In [50]:
from elasticsearch import Elasticsearch
import nltk
import math
import sys
import enchant
d = enchant.Dict("en_US")

In [229]:
es = Elasticsearch(['http://localhost:9200/'])
doc = {
        'size' : 301,
        'query': {
            'match_all' : {}
       }
   }
res = es.search(index='duc', doc_type='document', body=doc,scroll='1m')

In [201]:
def accept_sentence(sentence_divergent_dict):
    max_divergence = 0
    ret_sentence = ""
    for sentence in sentence_divergent_dict:
        if sentence_divergent_dict[sentence] > max_divergence:
            max_divergence = sentence_divergent_dict[sentence]
            ret_sentence = sentence
    return ret_sentence

In [230]:
summaries = {}
for item in res['hits']['hits']:
    text = item['_source']['doc_text']
    id_ = item['_source']['doc_id']
    sentences = text.split(".")
    text = nltk.tokenize.word_tokenize(text)
    for word in text:
        if not d.check(word):
            if len(word) < 4:
                text.remove(word)
    global_dist = nltk.FreqDist(text)
    i = 0
    summary = ""
    while i < 10 and len(sentences) != 0:
        sentence_divergence_dict = {}
        for sentence in sentences:
            merged_text = nltk.tokenize.word_tokenize(summary + sentence)
            merged_dist = nltk.FreqDist(merged_text)
            divergence = 0
            for word in global_dist:
                if word in merged_dist:
                    if merged_dist[word] != 0:
                        divergence += global_dist[word] * math.log(global_dist[word] / merged_dist[word])
            sentence_divergence_dict[sentence] = divergence
        sentence_to_add = accept_sentence(sentence_divergence_dict)
        summary += sentence_to_add + ". "
        if sentence_to_add in sentences:
            sentences.remove(sentence_to_add)
        i += 1
    summaries[id_] = summary

In [233]:
summaries['SJMN91-06143070 ']

';    Guerrillas suspected;  Most of the early speculation, however,focused on Gandhi\'s most hardened opponents in Tamil Nadu, the LiberationTigers of Tamil Eelam, a disciplined and violent guerrilla force that has usedTamil Nadu as a base in its campaign to win a separate state in Sri Lanka, theisland nation just off India\'s southeastern coast. ;   Socialist Janata Dal: The party was formed in November with Congress Partybacking by current prime minister Chandra Shekhar, who heads interimgovernment. " J. ;    Political parties;   Congress Party: Centrist party, led India\'s independence struggle againstBritish colonial rule.  The CIA is oftenblamed by India\'s conspiracy-minded political activists for a wide variety ofthe country\'s ills.  Has about195 Parliament seats.  There were no injuries. Assassinated during campaign stop Tuesday. At least 14 other people, including a senior police officer, died in theblast.  Supported by militant Hindu groups. '